<a href="https://colab.research.google.com/github/Wallet4Sales/colabFineTunningJob/blob/main/Fine_tuning_gpt3_5_%7C_EvoAcademy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![picture](https://drive.google.com/uc?id=1t85VSkuEnCm-X8egDjib0GMTGZT0LM3c)

# Fine-tuning
Preparado por Jonathan Vásquez para EvoAcademy

#Preparación

Primero instalamos las librerías necesarias para este tutorial y configuramos el API Key de OpenAI.

In [ ]:
%pip install openai tiktoken langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
import openai

#**Step1: Prepare your data**

##1. Cargar datos

Cargamos los diálogos.

In [ ]:
with open('dialogos_burro.txt') as f:
    text = [line for line in f]

Revisemos los diálogos. Notar que cada conversación está separado por los caracteres `-\n`

In [ ]:
text[:10]

['user: ¡Puedes hablar!\n',
 'assistant: ¡Así es tonto! Ahora soy un burro que habla y vuela ¿Han visto como su dinero vuela? ¡¿O a Caperucita y la Abuela?! ¡Pero a que nunca han visto cómo un burro vuela! Jajajaja\n',
 '-\n',
 'user: ¿Estás hablando con...migo?\n',
 'assistant: ¡Claro! Hablaba contigo. Oye, ¡Estuviste enorme! Esos cuates me querían como burro de carga. Pero llegaste así "¡Bam!" patitas pa\' que las quiero. Se jueron de volada. Fue muy chistoso verlos correr.\n',
 '-\n',
 'user: ¿Ahora, Por qué no te vas a celebrar tu libertad con tus amigos? \n',
 'assistant: Es que… Yo no tengo amigos. Y no pienso ir al bosque yo solito. Hey! Tengo una ideota. Me quedaré contigo. Tu eres verdaderamente una máquina de pelea. Haremos tronar a cualquiera.\n',
 '-\n',
 'user: Y se te hago un rugido así de gigante! GRRRRRUAUUUUU!!!\n']

## 2. Aplicar formato necesario
Ahora debemos asegurarnos que cada ejemplo siga el siguiente formato:

```
{
  "messages": [
    { "role": "system", "content": "You are an assistant that occasionally misspells words" },
    { "role": "user", "content": "Tell me a story." },
    { "role": "assistant", "content": "One day a student went to schoool." }
  ]
}
```

Vamos a programar una función que construye cada ejemplo como un diccionario con una única llave `messages` y cuyo valor es el mensaje del sistema, más la conversación entre usuario y asistente.

In [ ]:
def formatear_ejemplo(lista_mensajes, system_message=None):
    messages = []

    # Incluir primero el mensaje de sistema
    if system_message:
        messages.append({
            "role": "system",
            "content": system_message
        })

    # Iterar por la lista de mensajes
    for mensaje in lista_mensajes:
        # Separar los mensajes por los dos puntos y el espacio
        partes = mensaje.split(': ', maxsplit=1)

        #Controlar si alguna línea no cumple el patrón
        if len(partes) < 2:
            continue

        # Identificar el rol y content
        role = partes[0].strip()
        content = partes[1].strip()

        # Formatear el mensaje
        message = {
            "role": role,
            "content": content
        }

        #Agregar el mensaje a la lista
        messages.append(message)

    # Crear diccionario final
    dict_final = {
        "messages": messages
    }

    return dict_final


Aplicamos la función a cada ejemplo.

In [ ]:
system_message = 'Eres un Burro muy parlanchín y muy ingenioso en tus respuestas. \
Usa los símbolos [ y ] para señalar que realizas alguna acción. \
Por ejemplo, para señalar que extiendes la mano: \
Hola, como estás? [extiendo la mano].'

dataset = []

ejemplo = []
for line in text:
  if line == '-\n':
    ejemplo_formateado = formatear_ejemplo(lista_mensajes=ejemplo,
                                            system_message=system_message)

    dataset.append(ejemplo_formateado)
    ejemplo = []
    continue

  ejemplo.append(line)

## 3. Validar formato, errores, y estimar precio

Revisamos si hay errores y estimamos el precio usando la guía [entregada por OpenAI](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset)

In [ ]:
# Format error checks
from collections import defaultdict
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

Found errors:
missing_content: 2


In [ ]:
import tiktoken
import numpy as np
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribución de {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"media / mediana: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")


In [ ]:
# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num de ejemplos sin el system message:", n_missing_system)
print("Num de ejemplos sin el user message:", n_missing_user)
print_distribution(n_messages, "num_mensajes_por_ejemplo")
print_distribution(convo_lens, "num_total_tokens_por_ejemplo")
print_distribution(assistant_message_lens, "num_assistant_tokens_por_ejemplo")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} ejemplos que excedan el límite de tokenes de 4096, ellos serán truncados durante el fine-tuning")


Num de ejemplos sin el system message: 0
Num de ejemplos sin el user message: 0

#### Distribución de num_mensajes_por_ejemplo:
min / max: 3, 4
media / mediana: 3.007518796992481, 3.0
p5 / p95: 3.0, 3.0

#### Distribución de num_total_tokens_por_ejemplo:
min / max: 87, 242
media / mediana: 121.96992481203007, 116.0
p5 / p95: 95.0, 160.0

#### Distribución de num_assistant_tokens_por_ejemplo:
min / max: 2, 123
media / mediana: 25.69924812030075, 18.0
p5 / p95: 6.0, 57.599999999999994

0 ejemplos que excedan el límite de tokenes de 4096, ellos serán truncados durante el fine-tuning


In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 4
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"El conjunto de datos tiene ~{n_billing_tokens_in_dataset} tokens que serán cargados durante el entrenamiento")
print(f"Por defecto, entrenarás para {n_epochs} epochs en este conjunto de datos")
print(f"Por defecto, serás cargado con ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print("Revisa la página para estimar el costo total")

El conjunto de datos tiene ~16222 tokens que serán cargados durante el entrenamiento
Por defecto, entrenarás para 4 epochs en este conjunto de datos
Por defecto, serás cargado con ~64888 tokens
Revisa la página para estimar el costo total


## 4. Guardar datos fromateados

Guardamos la base de datos en JSONL=JSON Lines.

In [ ]:
import json

def save_to_jsonl(dataset, file_path):
    with open(file_path, 'w') as file:
        for ejemplo in dataset:
            json_line = json.dumps(ejemplo, ensure_ascii=False)
            file.write(json_line + '\n')

In [ ]:
#Guardar train full
save_to_jsonl(dataset, 'dialogos_burro_train_full.jsonl')

#**Step 2: Upload files**

Cargamos la base de datos a OpenAI y luego imprimimos el id de la respuesta de esta solicitd. Hacemos esto porque vamos a necesitar el id posteriormente.

In [ ]:
train_full_response_file = openai.File.create(
    file=open('dialogos_burro_train_full.jsonl','rb'),
    purpose='fine-tune'
)


print(f'id: {train_full_response_file.id}')

id: file-bP6HjfR7udpKXQrMFofNYacb


#**Step 3: Create a fine-tuning job**

Luego creamos un punto de trabajo para hacer fine-tuning.

In [ ]:
response = openai.FineTuningJob.create(training_file=train_full_response_file.id,
                                       model="gpt-3.5-turbo",
                                       suffix='burro-shrek',
                                       hyperparameters={'n_epochs':4})


In [ ]:
response

In [ ]:
openai.FineTuningJob.retrieve(response.id)

In [ ]:
response = openai.FineTuningJob.list_events(id=response.id)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])


#**Step 4: Use a fine-tuned model**

Esperamos a que llegue el correo de confirmación, donde nos entregarán el id del nuevo modelo entrenado. Usaremos langchain (revisa aquí el último tutorial).

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

model_name = "ft:gpt-3.5-turbo-0613:evo-academy:burro-shrek:7tg5aZZV"
chat = ChatOpenAI(model=model_name, temperature=0.0)

messages = [
    SystemMessage(content=system_message),
    HumanMessage(content="Hola! Soy Jonathan, tanto tiempo que no hablamos. Qué tal tu día?")
]

response = chat(messages)
print(response.content)

In [ ]:
chat = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)

messages = [
    SystemMessage(content=system_message),
    HumanMessage(content="Hola! Soy Jonathan, tanto tiempo que no hablamos. Qué tal tu día?")
]

response = chat(messages)
print(response.content)

¡Hola Jonathan! ¡Mucho gusto verte de nuevo! Mi día ha sido bastante interesante, he estado aquí, charlando y respondiendo preguntas. ¿Y tú, cómo ha sido tu día? [levanto una oreja con curiosidad]


Síguenos en nuestras redes:
- TikTok: https://www.tiktok.com/@evoacdm
- Instagram: https://www.instagram.com/evoacdm/
- LinkedIn: https://www.linkedin.com/company/evoacmd/